In [10]:
import pandas as pd
raw_path = "14100310.csv"  # adjust if different
lfs = pd.read_csv(raw_path, low_memory=False)
print(lfs.columns.tolist())
lfs.head()


['REF_DATE', 'GEO', 'DGUID', 'National Occupational Classification (NOC)', 'Statistics', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']


,REF_DATE,GEO,DGUID,National Occupational Classification (NOC),Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1987-01,Canada,2021A000011124,"Total employed, all occupations [00-95]",Estimate,Persons in thousands,428,thousands,3,v104166904,1.1.1,12102.0,NaN,NaN,NaN,1
1,1987-01,Canada,2021A000011124,"Total employed, all occupations [00-95]",Standard error of estimate,Persons in thousands,428,thousands,3,v104166905,1.1.2,NaN,..,NaN,NaN,1
2,1987-01,Canada,2021A000011124,"Total employed, all occupations [00-95]",Standard error of month-to-month change,Persons in thousands,428,thousands,3,v104166906,1.1.3,NaN,..,NaN,NaN,1
3,1987-01,Canada,2021A000011124,"Total employed, all occupations [00-95]",Standard error of year-over-year change,Persons in thousands,428,thousands,3,v104166907,1.1.4,NaN,..,NaN,NaN,1
4,1987-01,Canada,2021A000011124,"Management occupations [00,10,20,30,40,50,60,7...",Estimate,Persons in thousands,428,thousands,3,v104166908,1.2.1,1275.5,NaN,NaN,NaN,1


In [8]:
import re

df = pd.read_csv(raw_path, low_memory=False)

# Keep only the core series: employment "Estimate"
df = df[df["Statistics"].str.contains("Estimate", case=False, na=False)].copy()

# Dates → year
df["REF_DATE"] = pd.to_datetime(df["REF_DATE"], errors="coerce")
df["year"] = df["REF_DATE"].dt.year
df = df[(df["year"] >= 2015) & (df["year"] <= 2024)]

# Province codes
province_map = {
    "Canada": "National",
    "Newfoundland and Labrador": "NL", "Prince Edward Island": "PE",
    "Nova Scotia": "NS", "New Brunswick": "NB", "Quebec": "QC",
    "Ontario": "ON", "Manitoba": "MB", "Saskatchewan": "SK",
    "Alberta": "AB", "British Columbia": "BC",
    "Yukon": "YT", "Northwest Territories": "NT", "Nunavut": "NU",
}
df["province"] = df["GEO"].replace(province_map)

# Ensure numeric (StatCan VALUE is "persons in thousands")
df["employment_thousands"] = pd.to_numeric(df["VALUE"], errors="coerce")

# Monthly → Annual: use the MEAN of 12 months (seasonally adjusted series)
annual = (df.groupby(["year","province","National Occupational Classification (NOC)"], dropna=False)
            ["employment_thousands"].mean()
            .reset_index()
            .rename(columns={"National Occupational Classification (NOC)": "noc",
                             "employment_thousands": "employment"}))

# Optional: drop National if you’ll analyze provinces only
# annual = annual[annual["province"] != "National"].copy()

# Save clean file
out_path = " LFS_employment_by_occupation_2015_2024_clean.csv"
annual.to_csv(out_path, index=False)
print(f"[SAVED] {out_path}")

# Quick checks
print(annual.head())
print(annual.groupby("year")["employment"].sum().round(1).head())


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
[SAVED] data/clean/LFS_employment_by_occupation_2015_2024_clean.csv
   year  province                                                noc  \
0  2015  National  Business, finance and administration occupatio...   
1  2015  National      Health occupations, except management [31-33]   
2  2015  National  Management occupations [00,10,20,30,40,50,60,7...   
3  2015  National  Natural and applied sciences and related occup...   
4  2015  National  Natural resources, agriculture and related pro...   

    employment  
0  1387.270833  
1   646.987500  
2   800.729167  
3   681.925000  
4   184.558333  
year
2015    18016.0
2016    18135.5
2017    18541.5
2018    18878.3
2019    19236.2
Name: employment, dtype: float64
